In [1]:
import sys
sys.path.insert(0, '../utils')

import os
import cv2
import numpy as np
from typing import Dict, Tuple
from skimage import io
from dataGenerator import *

%reload_ext autoreload
%autoreload 2

In [2]:
DATA_BANK_DIRECTORY

'/home/mark/DataBank/probav_data/'

In [3]:
# Load dataset
imgAllSets = generateImageSet(isTrainData=True, NIR=True)

# Remove outliers
imgAllSets = removeImageWithOutlierPixels(imageSet=imgAllSets, threshold=60000, isTrainData=True)

# Upscale
imgAllSetsUpScaled = upsampleImages(imageSets=imgAllSets, scale=3)  # 128x128 -> 384x384

# Convert dataset to numpy array
output = imageSetToNumpyArrayHelper(imgAllSetsUpScaled, imgAllSets, isGrayScale=True, isNHWC=False)

# Get shifts and trim dataset based on shift threshold
# Read more about this shifts happening in LR Images here
# https://kelvins.esa.int/proba-v-super-resolution/data/
saveArrays(output, DATA_BANK_DIRECTORY_PREPROCESSING_CHKPT, 'NIR')


correctShiftsFromSavedArrays(os.path.join(DATA_BANK_DIRECTORY_PREPROCESSING_CHKPT,
                                              'numpyArrays', 'NIR'),
                             os.path.join(DATA_BANK_DIRECTORY_PREPROCESSING_CHKPT,
                                              'correctedShifts', 'NIR'),
                             output['names'], upsampleScale=1)

srcFolder = os.path.join(DATA_BANK_DIRECTORY_PREPROCESSING_CHKPT, 'correctedShifts', 'NIR')
dstFolder = os.path.join(DATA_BANK_DIRECTORY_PREPROCESSING_CHKPT, 'patches', 'upsample', 'NIR')

generatePatchDatasetFromSavedFile(srcFolder, dstFolder, output['names'], useUpsample=True,
                                      patchSize=96, thresholdPatchesPerImgSet=20, thresholdClarityLR=0.7,
                                      thresholdClarityHR=0.85)

[ INFO ] Correcting shifts in imagesets        : 100%|██████████| 562/562 [05:45<00:00,  1.63it/s]
[ INFO ] Upscaling LowRes images               :   0%|          | 0/561 [00:00<?, ?it/s]

ValueError: need at least one array to stack